# Excel/CSV → JSON (Title, Abstract, EID)

This notebook loads a Scopus export (Excel or CSV), extracts **Title**, **Abstract**, **EID**, and produces a JSON array:

```json
[
  {"title": "...", "abstract": "...", "eid": "..."},
  ...
]
```

It also writes a CSV copy for easier inspection.

In [ ]:
# Install (only if needed)
# !pip install pandas openpyxl

In [1]:
import pandas as pd
import json
from pathlib import Path

# --- CONFIG ---
# INPUT_PATH = Path("scopus-tytuły-czysty.xlsx")  # or Path("scopus-tytuły-czysty.csv")
# OUTPUT_JSON = Path("records.json")
# OUTPUT_CSV  = Path("records_for_userscript.csv")

INPUT_PATH = Path("wyniki_ai.xlsx")  # or Path("scopus-tytuły-czysty.csv")
OUTPUT_JSON = Path("wyniki_ai_result.json")
OUTPUT_CSV  = Path("wyniki_ai_result.csv")

def pick_column(df, variants):
    cols = {c.lower().strip(): c for c in df.columns}
    for v in variants:
        key = v.lower().strip()
        if key in cols:
            return cols[key]
    return None

def load_table(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"Input file not found: {path}")
    if path.suffix.lower() in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    elif path.suffix.lower() == ".csv":
        return pd.read_csv(path)
    else:
        raise ValueError("Unsupported file extension. Use .xlsx, .xls, or .csv")

def to_records(df: pd.DataFrame):
    c_title = pick_column(df, ["Title", "Tytuł", "title", "tytuł"])
    c_abs   = pick_column(df, ["Abstract", "Streszczenie", "abstract", "streszczenie"])
    c_eid   = pick_column(df, ["EID", "Scopus EID", "eid", "scopus eid"])

    if not c_title or not c_abs:
        raise ValueError(f"Missing required columns. Found: {list(df.columns)}. "
                         f"Need: Title/Tytuł and Abstract/Streszczenie (EID optional).")

    out = []
    for _, r in df.iterrows():
        title = str(r.get(c_title, "")).strip()
        abstract = str(r.get(c_abs, "")).strip()
        eid = str(r.get(c_eid, "")).strip() if c_eid else ""
        if title or abstract:
            out.append({"title": title, "abstract": abstract, "eid": eid})
    return out

df = load_table(INPUT_PATH)
records = to_records(df)

print({
    "rows_in_source": len(df),
    "exported_records": len(records),
    "missing_abstract_rows": sum(1 for r in records if not r["abstract"])
})

# Save outputs
OUTPUT_JSON.write_text(json.dumps(records, ensure_ascii=False, indent=2), encoding="utf-8")
pd.DataFrame(records).to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print("Written:", OUTPUT_JSON.resolve())
print("Written:", OUTPUT_CSV.resolve())

{'rows_in_source': 57, 'exported_records': 57, 'missing_abstract_rows': 0}
Written: C:\Users\Marcin Kutrzynski\Documents\JupyterNotebooks\PRISMA tests\wyniki_ai_result.json
Written: C:\Users\Marcin Kutrzynski\Documents\JupyterNotebooks\PRISMA tests\wyniki_ai_result.csv


## Optional: quick preview
Display first rows to verify column mapping.

In [ ]:
import pandas as pd
pd.DataFrame(records)[:10]

## Optional: serve `records.json` locally
If you want to load the JSON from your Tampermonkey userscript via `fetch('http://localhost:8000/records.json')`, run the cell below. It will start a simple HTTP server in the current working directory.

> Stop the server by interrupting the kernel.

In [ ]:
import http.server, socketserver, threading, os

PORT = 8000
DIR = os.getcwd()  # ensure records.json is in this directory
os.chdir(DIR)

def run_server():
    Handler = http.server.SimpleHTTPRequestHandler
    with socketserver.TCPServer(("127.0.0.1", PORT), Handler) as httpd:
        print(f"Serving {DIR} at http://127.0.0.1:{PORT}")
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            pass

thread = threading.Thread(target=run_server, daemon=True)
thread.start()
print("Server started. Open http://127.0.0.1:8000/records.json")